In [1]:
import time
import os
import numpy as np
from dotmap import DotMap
import traceback
import argparse
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

from verifai.samplers.scenic_sampler import ScenicSampler
from verifai.scenic_server import ScenicServer
from verifai.falsifier import generic_falsifier
from verifai.monitor import specification_monitor
from verifai.falsifier import generic_falsifier

/Users/kesav/Documents/Carla/Scenic/src/scenic/core/errors.py:160: UserWarning: unable to install sys.excepthook to format Scenic backtraces
  warnings.warn('unable to install sys.excepthook to format Scenic backtraces')
/Users/kesav/Library/Caches/pypoetry/virtualenvs/verifai-vzZ5ZzH9-py3.8/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


In [2]:
class distance(specification_monitor):
    """
    Single-objective specification. This monitor takes a
    minimum over the distances from each vehicle. If the ego vehicle is less than 5 meters
    away from any vehicle at any point, a counterexample is returned.
    """
    def __init__(self):
        def specification(simulation):
            min_dist = np.inf
            for timestep in simulation.result.trajectory:
                num_vehicles = len(timestep)
                if num_vehicles < 2:
                    continue
                ego_pos = timestep[0]
                for other_pos in timestep[1:]:
                    dist = ego_pos.distanceTo(other_pos)
                    min_dist = min(min_dist, dist)
            rho = min_dist - 5
            return rho
        
        super().__init__(specification)

In [3]:
def run_experiment(path, parallel=False, model=None,
                   sampler_type=None, headless=False, num_workers=5, scenario=None):
    """
    Runs a single falsification experiment.

    Arguments:
        path: Path to Scenic script to be run.
        parallel: Whether or not to enable parallelism.
        model: Which simulator model to use (e.g. carla, lgsvl, newtonian, etc.)
        sampler_type: Which VerifAI sampelr to use (e.g. halton, scenic, ce, mab, etc.)
        headless: Whether or not to display each simulation.
        num_workers: Number of parallel workers. Only used if parallel is true.
    """
    model = f'scenic.simulators.{model}.model' if model else None
    params = {'verifaiSamplerType': sampler_type, 'scenario': scenario} if sampler_type else {}
    params['render'] = not headless
    if model:
        params['model'] = model
    sampler = ScenicSampler.fromScenario(path, **params)
    num_objectives = sampler.scenario.params.get('N', 1)
    multi = num_objectives > 1
    falsifier_params = DotMap(
        n_iters=5,
        save_error_table=True,
        save_safe_table=True,
        max_time=None,
    )
    server_options = DotMap(maxSteps=200, verbosity=0)
    monitor = distance()

    falsifier_cls = generic_parallel_falsifier if parallel else generic_falsifier
    
    falsifier = falsifier_cls(sampler=sampler, falsifier_params=falsifier_params,
                                  server_class=ScenicServer,
                                  server_options=server_options,
                                  monitor=monitor, scenic_path=path,
                                  scenario_params=params, num_workers=num_workers)
    t0 = time.time()
    print('Running falsifier')
    falsifier.run_falsifier()
    t = time.time() - t0
    print()
    print(f'Generated {len(falsifier.samples)} samples in {t} seconds with {falsifier.num_workers} workers')
    print(f'Number of counterexamples: {len(falsifier.error_table.table)}')
    if not parallel:
        print(f'Sampling time: {falsifier.total_sample_time}')
        print(f'Simulation time: {falsifier.total_simulate_time}')
    print(f'Confidence interval: {falsifier.get_confidence_interval()}')
    if multi:
        print(f'Counterexamples found: {falsifier.server.sampler.scenario.externalSampler.sampler.domainSampler.split_sampler.samplers[0].counterexamples.keys()}')
    return falsifier

In [5]:
# pass the argument headless=True to run without displaying Newtonian simulator.
falsifier = run_experiment('uberCrashNewton.scenic')

/Users/kesav/Documents/Carla/Scenic/src/scenic/formats/opendrive/xodr_parser.py:1429: OpenDriveWarning: road 575 has a lane section of length 9.999999690535333e-07; skipping it
  warn(f'road {road.id_} has a lane section of length {l}; skipping it')
/Users/kesav/Documents/Carla/Scenic/src/scenic/formats/opendrive/xodr_parser.py:677: OpenDriveWarning: ignoring shoulder in the middle of road 5
  warn(f'ignoring {name} in the middle of road {self.id_}')
/Users/kesav/Documents/Carla/Scenic/src/scenic/formats/opendrive/xodr_parser.py:677: OpenDriveWarning: ignoring shoulder in the middle of road 6
  warn(f'ignoring {name} in the middle of road {self.id_}')
/Users/kesav/Documents/Carla/Scenic/src/scenic/formats/opendrive/xodr_parser.py:677: OpenDriveWarning: ignoring shoulder in the middle of road 7
  warn(f'ignoring {name} in the middle of road {self.id_}')
/Users/kesav/Documents/Carla/Scenic/src/scenic/formats/opendrive/xodr_parser.py:677: OpenDriveWarning: ignoring shoulder in the middle 

/Users/kesav/Documents/Carla/Scenic/src/scenic/formats/opendrive/xodr_parser.py:677: OpenDriveWarning: ignoring shoulder in the middle of road 1470
  warn(f'ignoring {name} in the middle of road {self.id_}')
/Users/kesav/Documents/Carla/Scenic/src/scenic/formats/opendrive/xodr_parser.py:677: OpenDriveWarning: ignoring sidewalk in the middle of road 1566
  warn(f'ignoring {name} in the middle of road {self.id_}')
/Users/kesav/Documents/Carla/Scenic/src/scenic/formats/opendrive/xodr_parser.py:677: OpenDriveWarning: ignoring shoulder in the middle of road 1566
  warn(f'ignoring {name} in the middle of road {self.id_}')
/Users/kesav/Documents/Carla/Scenic/src/scenic/formats/opendrive/xodr_parser.py:677: OpenDriveWarning: ignoring shoulder in the middle of road 1577
  warn(f'ignoring {name} in the middle of road {self.id_}')
/Users/kesav/Documents/Carla/Scenic/src/scenic/formats/opendrive/xodr_parser.py:677: OpenDriveWarning: ignoring shoulder in the middle of road 1624
  warn(f'ignoring {n

sampler type is ce
Running falsifier


100% (5 of 5) |##########################| Elapsed Time: 0:00:32 ETA:  00:00:00


Generated 5 samples in 41.42051291465759 seconds with 1 workers
Number of counterexamples: 3
Sampling time: 0.19050073623657227
Simulation time: 41.21137857437134
Confidence interval: (0.14663279963467318, 0.9472550494736831)


In [6]:
df = pd.concat([falsifier.error_table.table, falsifier.safe_table.table])

In [7]:
"""
Shows the sampled parameters and specification satisfaction value rho.
If rho < 0, that simulation corresponds to a counterexample.
"""
df.head()

,point.objects.object0.color[0],point.objects.object0.color[1],point.objects.object0.color[2],point.objects.object0.heading,point.objects.object0.position[0],point.objects.object0.position[1],point.objects.object1.color[0],point.objects.object1.color[1],point.objects.object1.color[2],point.objects.object1.heading,point.objects.object1.position[0],point.objects.object1.position[1],point.params.DISTANCE_TO_INTERSECTION,point.params.HESITATION_TIME,point.params.UBER_SPEED,rho
0,1.000000,1.000000,1.000000,1.549377,111.935078,135.874351,0.730548,0.730548,0.730548,-1.592216,70.764562,133.255547,-15.754982,7.417403,12.954932,-1.080432
1,0.165874,0.165874,0.165874,1.549377,111.174199,135.890652,0.700592,0.697972,0.693802,-1.592216,70.764562,133.255547,-14.993928,7.485726,13.094212,-1.408053
2,0.795624,0.279637,0.263801,1.549377,111.603125,135.881463,0.970783,0.970783,0.970783,-1.592216,70.764562,133.255547,-15.422952,5.087528,12.280443,-0.050327
0,0.908148,0.908148,0.908148,3.124310,82.615486,161.441737,0.328235,0.328235,0.328235,-0.035328,84.778026,122.005521,-13.985478,2.271541,11.818209,1.280460
1,0.853464,0.853464,0.853464,-0.035328,84.222204,106.278731,0.742027,0.742027,0.742027,3.106265,82.175580,147.463325,-15.736609,1.377593,16.187258,3.021294
